# Clase 06 - Analisis y visualizacion: Establecimientos Educacionales

Profesor: **Fernando Becerra**, f.becerra@udd.cl, [www.fernandobecerra.com](www.fernandobecerra.com)

En esta clase analizaremos un caso de estudio de comienzo a fin, en el cual la idea es explorar los dats a través de la visualización de información para poder delinear una narrativa. En particular, analizaremos datos de los establedicmientos educacionales de la Región Metropolitana, aunque la aplicación del análisis no tiene porqué esta acotado a esa área.

Este ejercicio fue inspirado por una visualización que vi en [Twitter](https://twitter.com/jcovarrubia/status/1539440850257727489). Recuerden que siempre es bueno seguir gente que trabaje en datos y estar atento a lo que ell@s están haciendo. En nuestro caso no haremos exactamente el mismo proceso, sino que lo complementaremos con otras bases de datos para hacer un análisis más completo.

Lo primero es lo primero: importar lo típico

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Como dice el tuit, los datos son sacados del [portal de datos abiertos del Ministerio de Educación](https://datosabiertos.mineduc.cl).  Si nos vamos a "Estudiantes y Párvulos" y después ["Matrícula por estudiante"](https://datosabiertos.mineduc.cl/matricula-por-estudiante-2/), podemos acceder al archivo en formato `zip`. Este archivo incluye documentos de texto que nos ayudan con los nombres de las columnas y los valores que cada una puede tomar.

A nosotr@s nos interesa cargar el archivos `csv` que viene dentro y empezar a ver qué información tiene.

In [ ]:
!ls ../../datos/educacion/Matricula-por-estudiante-2021/20210913_Matrícula_unica_2021_20210430_WEB.CSV

In [ ]:
df = pd.read_csv('../../datos/educacion/Matricula-por-estudiante-2021/20210913_Matrícula_unica_2021_20210430_WEB.CSV',
                 sep=';',
                 low_memory=False)
df.head()


Lo primero que haremos para hacer el análisis un poco más fácil, es restringir el análisis a la Región Metropolitana. Para eso, eligiremos sólo los establecimientos y alumn@s que se ubican en dicha región, y filtraremos los datos en base a eso.

In [ ]:
df_rm = df[(df['COD_REG_RBD'] == 13) & (df['COD_REG_ALU'] == 13)].reset_index()
df_rm.head()
# df_rm = df[(df['COD_PRO_RBD'] == 131) & (df['COD_COM_ALU'].isin(comunas_santiago))].reset_index()

Para empezar a familiarizarnos con los datos y con ayuda de los documentos adjuntos, empezamos a ver de qué se trata cada columna y como vamos a trabajar con ellas. Por ejemplo, sería bueno saber el número total de establecimientos luego de haberlos filtrados.

In [ ]:
len(pd.unique(df_rm['RBD']))

## Parte 1: Movimiento general de la matrícula en la Región Metropolitana

Para comenzar el análisis, nos inspiraremos en un ejemplo visto en las clases anteriores: la encuesta origen-destino. Veremos la movilidad de cada estudiante en base a la comuna donde vive y la comuna donde estudia. Para eso reutilizaremos código de ese ejemplo para calcularel flujo desde una comuna a otra,

In [ ]:
from sklearn.preprocessing import normalize

def normalize_rows(df):
    df = pd.DataFrame(normalize(df, norm='l1'), index=df.index, columns=df.columns)
    return df

def normalize_columns(df):
    df = pd.DataFrame(normalize(df, norm='l1', axis=0), index=df.index, columns=df.columns)
    return df

In [ ]:
flujos = (
    df_rm.groupby(['NOM_COM_RBD', 'NOM_COM_ALU'])
          .agg(n_matriculas=('NOM_COM_ALU', 'count'))
          ['n_matriculas'].unstack(fill_value=0)
          .pipe(normalize_rows)
)
flujos.head()

Una de mis visualizaciones favoritas es el heat map, por lo que ocuparemos ese gráfico para visualizar esta red.

In [ ]:
plt.figure(figsize=(14, 14))
sns.heatmap(flujos, cmap='magma_r', square=True, linewidths=1, 
            cbar_kws={'shrink': 0.8, 'label': 'Fracción de matrículas'})
plt.xlabel('Comuna Establecimiento')
plt.ylabel('Comuna Estudiantes')
plt.show()

## Parte 2: Origen de la matrícula de cada establecimiento

La visualización anterior nos da una idea general de como se mueven los estudiantes desde una comuna a otra, pero ahora queremos ir más al detalle. Comenzar con una vista general para luego ir a lo específico es una buena forma de hacer la información más digerible.

En este paso calcularemos la cantidad de matrículas de cada colegio por comuna de origen. 

In [ ]:
establecimientos = (
    df_rm.groupby(['RBD','NOM_RBD', 'COD_COM_ALU'])
        .agg(n_matriculas=('COD_COM_ALU', 'count'))
)

establecimientos.head()

Para hacerlo comparables entre distintos establecimientos educacionales, lo transformaremos a porcentaje de matrícula.

In [ ]:
establecimientos['n_percent'] = establecimientos['n_matriculas'] / establecimientos.groupby('NOM_RBD')['n_matriculas'].transform('sum')
establecimientos.head(3)

In [ ]:
establecimientos = establecimientos.reset_index()

¡Ahora es momento de empezar a mapear! Para eso importamos `geopandas`

In [ ]:
import geopandas as gpd

Y cargamos los arhivos `shp` sacados desde la [Biblioteca del Congreso Nacional](https://www.bcn.cl/siit/mapas_vectoriales/index_html).

In [ ]:
comunas = gpd.read_file('../../../datos/bcn/Comunas/comunas.shp')
comunas.head()

Exploramos las regiones disponible para poder filtrarlas

In [ ]:
pd.unique(comunas['Region'])

Y ahora filtramos los datos

In [ ]:
comunas_rm = comunas[comunas['Region'] == 'Región Metropolitana de Santiago'].reset_index()
comunas_rm.head()

Para ver que todo esté en orden, lo vamos a graficar primero.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

comunas_rm.plot(ax=ax)

plt.show()

Ahora definiremos una función que reciba el nombre de un establecimiento y grafique el porcentaje de estudiantes matriculados por comuna.

In [ ]:
def plot_establecimiento(establecimiento, ax):

    filtered_df = establecimientos[establecimientos['NOM_RBD'] == establecimiento]
    establecimiento_df = (
        pd.merge(comunas_rm,
            filtered_df.set_index('COD_COM_ALU'),
            left_on='cod_comuna',
            right_index=True,
            how='left')
        .fillna(0)
    )

    establecimiento_df.plot(ax=ax,
                            column='n_percent', 
                            cmap='BuPu',
                            linewidth=0.2,)
    
    ax.set_title(establecimiento)


Ahora veamos que establecimientos están disponibles.

In [ ]:
print(list(pd.unique(establecimientos['NOM_RBD'])))

Empecemos a probar algunos.

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,8))

plot_establecimiento('LICEO JAVIERA CARRERA', ax[0])
plot_establecimiento('COLEGIO SAINT GEORGE S COLLEGE', ax[1])

ax[0].set_axis_off()
ax[1].set_axis_off()

# ax[0].set_xlim(-7.92e6, -7.83e6)
# ax[0].set_ylim(-4e6, -3.91e6)
# ax[1].set_xlim(-7.92e6, -7.83e6)
# ax[1].set_ylim(-4e6, -3.91e6)

plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,8))

plot_establecimiento('LICEO INSTITUTO NACIONAL', ax[0])
plot_establecimiento('COLEGIO THE GRANGE SCHOOL', ax[1])

ax[0].set_axis_off()
ax[1].set_axis_off()

plt.show()

Este tipo de visualización tiene mucho potencial una vez que se identifiquen establecimientos interesantes. En principio se podría hacer una infografía o un póster usando *small multiples*, pero eso queda como trabajo extra.

## Parte 3: Creando un índice de pobreza por establecimiento

Ahora que tenemos el porcentaje de estudiantes que vienen de distintas comunas, podríamos ocupar datos comunales para derivar otros indicadores. Por ejemplo, podríamos buscar datos de tasa de pobreza por comuna y ver cómo usar es para caracterizar la población de cada establecimiento educacional.

Los datos de tasas de pobreza comunales pueden ser sacados de la [Encuesta Casen](http://observatorio.ministeriodesarrollosocial.gob.cl/encuesta-casen-en-pandemia-2020).

In [ ]:
casen = pd.read_excel('../../../datos/casen/Estimaciones_de_Tasa_de_Pobreza_por_Ingresos_por_Comunas_2020.xlsx', skiprows=2)
casen.head()

Filtramos sólo los datos de la Región Metropolitana.

In [ ]:
casen_rm = casen[casen['Región'] == 'XIII Metropolitana de Santiago'].reset_index()
casen_rm.head()

Aquí la idea es tener un indicador de situación socioeconómica por establecimiento educacional. Para eso, calcularemos un promedio usando como peso el porcentaje de estudiantes de esa comuna.

In [ ]:
def weighted_mean(df, value='Porcentaje de personas en situación de pobreza por ingresos 2020', weight='n_percent'):
    weighted_sum = (df[value] * df[weight]).sum()
    return weighted_sum

Hay que tener cuidado como se manejan los datos para poder mezclarlos y hacer operaciones en ellos.

In [ ]:
tasa_pobreza = (
    pd.merge(establecimientos,
             casen_rm.set_index('Código'),
             left_on='COD_COM_ALU',
             right_index=True)
    .groupby(['RBD'])
    .apply(weighted_mean)
    .reset_index()
)
tasa_pobreza.columns = ['RBD', 'TASA_POBREZA']
tasa_pobreza.head()

Información adicional que nos va a servir en nuestro análisis es el tipo de subvención que tiene cada establecimiento: municipal, particular subvencionado o particular pagado. Por lo que agregamos esa columna a nuestro dataframe.

In [ ]:
depe = (
    df_rm.groupby('RBD')
        .first()
        .reset_index()[['RBD', 'NOM_RBD', 'COD_DEPE2']]
)
depe.head()

Y lo mezclamos con nuestros datos

In [ ]:
rbd_pobreza = tasa_pobreza.merge(depe)
rbd_pobreza.head()

Ahora graficamos las distribuciones de los establecimientos en base al índice de pobreza que recién calculamos. Para hacer el análisis más interesante, lo dividiremos según el tipo de subvención que reciba el establecimiento.

In [ ]:
labels = ['Municipal', 'Particular Subvencionado', 'Particular Pagado']
colors = ['#377eb8', '#ff7f00', '#4daf4a']

fig, ax = plt.subplots(1,2,figsize=(16,6))

for depe in range(1, 4):
    filtered = rbd_pobreza[rbd_pobreza['COD_DEPE2'] == depe]
    filtered.hist(column='TASA_POBREZA', ax=ax[0], alpha=0.5, label=labels[depe-1], bins=100, color=colors[depe-1])
    
    filtered = rbd_pobreza[rbd_pobreza['COD_DEPE2'] == depe]
    filtered['TASA_POBREZA'].plot.kde(ax=ax[1], alpha=0.5, label=labels[depe-1], color=colors[depe-1])
    
ax[0].legend()
ax[1].legend()
plt.show()

Otra de mis visualizaciones favoritas para representar distribuciones es el `beeswarm`. Afortunadamente, `seaborn` trae una función que nos permite crearla de una forma sencilla.

In [ ]:
labels = ['Municipal', 'Particular Subvencionado', 'Particular Pagado']
colors = ['#377eb8', '#ff7f00', '#4daf4a']

# fig, ax = plt.subplots(3,1,figsize=(10,6), sharex=True)

for depe in range(1, 4):
    filtered = rbd_pobreza[rbd_pobreza['COD_DEPE2'] == depe]
    sns.swarmplot(x = filtered['TASA_POBREZA']) #, ax=ax[depe-1], 
                  #alpha=0.9, label=labels[depe-1], size=2, color=colors[depe-1])
    
#     ax[depe-1].set_xlim(0.02,0.16)
#     ax[depe-1].set_yticklabels([labels[depe-1]])
#     ax[depe-1].text(0, 0, labels[depe-1], color=colors[depe-1])

#     if (depe == 3):
#         for key, spine in ax[depe-1].spines.items():
#             spine.set_visible(False)
#         ax[depe-1].spines['left'].set_visible(False)
#         ax[depe-1].spines['right'].set_visible(False)
#         ax[depe-1].xaxis.label.set_color('#525252')
#         ax[depe-1].tick_params(axis='x', colors='#525252')
#         ax[depe-1].set_xlabel('Tasa pobreza')
#         ax[depe-1].set_yticks([])
#     else:
#         ax[depe-1].set_axis_off()

# fig.suptitle('La mayoría de los colegios particulares subvencionados se enfocan en estudiantes de clase media y media-baja', x=0.005, y=0.98, ha='left')
fig.tight_layout()
plt.subplots_adjust(hspace=0)
plt.show()

Encontrar los comandos correctos para borrar `ticks`, `ticklabels` y `spines` puede tomar tiempo, así que dejo [este enlace](https://stackabuse.com/matplotlib-turn-off-axis-spines-ticklabels-axislabels-grid/) que los puede ayudar a personalizar sus gráficos. Igual que en la parte anterior, hay mucho que se le puede agregar y mejorar a esta visualización, por lo que eso también queda como ejercicio para cada uno de ustedes.

## Parte 4: Comparación índice de pobreza con resultados SIMCE

Para poder comparar pobreza con resultados en pruebas estandarizadas, debemos cargar nuevos datos. Esta vez, los sacamos de [la base de datos del SIMCE](https://informacionestadistica.agenciaeducacion.cl/#/bases).

In [ ]:
simce = pd.read_excel('../../../datos/educacion/Simce8b2019_publicas_web/Archivos XLS (XLSX)/simce8b2019_rbd.xlsx')
simce.head()

Y filtramos sólo los establecimientos con los cuales hemos estado trabajando hasta el momento.

In [ ]:
simce_rm = simce[simce['rbd'].isin(rbd_pobreza['RBD'])]
simce_rm.head()


Exploramos las columnas disponibles para saber cuáles nos servirán.

In [ ]:
simce_rm.columns

Definimos las que tengan puntajes de las pruebas para usarlas después.

In [ ]:
pruebas_simce = ['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']
simce_rm = simce_rm[['rbd'] + pruebas_simce]
simce_rm.columns = ['RBD'] + pruebas_simce
simce_rm.head()

Los mezclamos con nuestro dataset

In [ ]:
simce_pobreza = rbd_pobreza.merge(simce_rm)
simce_pobreza.head()

Y graficamos para ver que relación hay entre esas variables

In [ ]:
fig, ax = plt.subplots(1,len(pruebas_simce), figsize=(14,4))
xmin = 0.02
xmax = 0.15


for idx, col in enumerate(pruebas_simce):
    simce_pobreza.plot.scatter('TASA_POBREZA', col, ax=ax[idx], alpha=0.2)
    ax[idx].set_xlim(xmin, xmax)
    ax[idx].set_ylim(150, 380)
    
plt.show()

Ahí se ve algo interesante (y lamentablemente esperable)... ¿hay algun tipo de relación? Tratemos de ajustar un modelo lineal para ver que sale. Para eso ocuparemos el paquete `scipy`.

In [ ]:
import scipy as sp

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,4))
xmin = 0.02
xmax = 0.15

for idx, col in enumerate(['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']):
    simce_pobreza.plot.scatter('TASA_POBREZA', col, ax=ax[idx], alpha=0.2)
    b, a, r, p, std = sp.stats.linregress(simce_pobreza['TASA_POBREZA'], simce_pobreza[col])
    x = np.linspace(xmin, xmax, 100)
    ax[idx].plot(x, a + b * x, color='#984ea3', lw=1.5)
    ax[idx].text(0.12, 365, 'R = {:.2f}'.format(r))
    ax[idx].set_xlim(xmin, xmax)
    ax[idx].set_ylim(150, 380)
    
plt.show()

La línea no se ve porque hay `nan` dentro del dataframe, así que nos tenemos que deshacer de ellos para que funcione.

In [ ]:
simce_pobreza_clean = simce_pobreza.dropna().sort_values(by='TASA_POBREZA')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,4))
xmin = 0.02
xmax = 0.15

for idx, col in enumerate(['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']):
    simce_pobreza_clean.plot.scatter('TASA_POBREZA', col, ax=ax[idx], alpha=0.2)
    b, a, r, p, std = sp.stats.linregress(simce_pobreza_clean['TASA_POBREZA'], simce_pobreza_clean[col])
    x = np.linspace(xmin, xmax, 100)
    ax[idx].plot(x, a + b * x, color='#984ea3', lw=1.5)
    ax[idx].text(0.12, 365, 'R = {:.2f}'.format(r))
    ax[idx].set_xlim(xmin, xmax)
    ax[idx].set_ylim(150, 380)
    
plt.show()

Ahora, sería interesante hacer este mismo gráfico pero diferenciado por tipo de subvención.

In [ ]:
labels = ['Municipal', 'Particular Subvencionado', 'Particular Pagado']
pruebas_simce = ['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']

fig, ax = plt.subplots(1, len(pruebas_simce), figsize=(14,4))
xmin = 0.02
xmax = 0.15

for idx, col in enumerate(pruebas_simce):
    for depe, label in enumerate(labels):
        depe_simce = simce_pobreza_clean[simce_pobreza_clean['COD_DEPE2'] == depe + 1]
        depe_simce.plot.scatter('TASA_POBREZA', col, ax=ax[idx], alpha=0.2, color=colors[depe])
        
        b, a, r, p, std = sp.stats.linregress(depe_simce['TASA_POBREZA'], depe_simce[col])
        x = np.linspace(xmin, xmax, 100)
        ax[idx].plot(x, a + b * x, color=colors[depe], lw=1.5)

        ax[idx].set_xlim(xmin, xmax)
        ax[idx].set_ylim(150, 380)

fig.tight_layout()
plt.show()

Y sería mejor aún si los separamos en distintos gráficos para ver todo más claramente.

In [ ]:
labels = ['Municipal', 'Particular Subvencionado', 'Particular Pagado']
pruebas_simce = ['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']

fig, ax = plt.subplots(len(labels), len(pruebas_simce), figsize=(14,12))
xmin = 0.02
xmax = 0.15

for depe, label in enumerate(labels):
    for idx, col in enumerate(pruebas_simce):
        depe_simce = simce_pobreza_clean[simce_pobreza_clean['COD_DEPE2'] == depe + 1]
        depe_simce.plot.scatter('TASA_POBREZA', col, ax=ax[depe][idx], alpha=0.2, color=colors[depe])
        b, a, r, p, std = sp.stats.linregress(depe_simce['TASA_POBREZA'], depe_simce[col])
        x = np.linspace(xmin, xmax, 100)
        ax[depe][idx].plot(x, a + b * x, color='#984ea3', lw=1.5)
        ax[depe][idx].text(0.12, 365, 'R = {:.2f}'.format(r))
        ax[depe][idx].set_xlim(xmin, xmax)
        ax[depe][idx].set_ylim(150, 380)

fig.tight_layout()
plt.show()

Recién ahora me dí cuenta de algo que no hemos considerado: el total de estudiantes por establecimiento.

In [ ]:
n_total = (
    df_rm.groupby(['RBD'])
        .agg(n_total=('COD_COM_ALU', 'count'))
        .reset_index()
)

n_total.head()

Y lo mezclamos con nuestro dataframe.

In [ ]:
simce_pobreza_total = simce_pobreza_clean.merge(n_total).dropna()
simce_pobreza_total.head()

Lamentablemente el método de `scipy` que ocupamos para ajustar el model lineal no acepta pesos como parámetros, por lo que tenemos que ocupar otro paquete llamado `sklearn`.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
labels = ['Municipal', 'Particular Subvencionado', 'Particular Pagado']
pruebas_simce = ['prom_lect8b_rbd', 'prom_soc8b_rbd', 'prom_mate8b_rbd']

fig, ax = plt.subplots(len(labels), len(pruebas_simce), figsize=(14,12))
xmin = 0.02
xmax = 0.15

for depe, label in enumerate(labels):
    for idx, col in enumerate(pruebas_simce):
        depe_simce = simce_pobreza_total[simce_pobreza_total['COD_DEPE2'] == depe + 1]
        depe_simce.plot.scatter('TASA_POBREZA', col, ax=ax[depe][idx], alpha=0.2, color=colors[depe])
        
        linreg = LinearRegression()
        linreg.fit(depe_simce[['TASA_POBREZA']], depe_simce[[col]], depe_simce['n_total'].to_numpy())
        r = linreg.score(depe_simce[['TASA_POBREZA']], depe_simce[[col]], depe_simce['n_total'].to_numpy())
        x = pd.DataFrame(np.linspace(xmin, xmax, 100), columns=['TASA_POBREZA'])
        x['prediction'] = linreg.predict(x[['TASA_POBREZA']])
        x.plot(x='TASA_POBREZA', y='prediction', color='#984ea3', lw=1.5, ax=ax[depe][idx])
        ax[depe][idx].text(0.12, 365, 'R = {:.2f}'.format(r))
        ax[depe][idx].set_xlim(xmin, xmax)
        ax[depe][idx].set_ylim(150, 380)

fig.tight_layout()
plt.show()

Hasta acá ya tenemos todo lo necesario para armar una historia, con una narrativa clara y una intención de qué es lo que queremos comunicar. Ahora hay que pasar harto tiempo mejorando los gráficos para que se ven mejor.